# **K-Compress: Image Compression using K-Means**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import time
import imageio

In [2]:
def euclidean_distance(sample, centroids):

    return np.sqrt(np.sum((sample - centroids)**2, axis=1))

In [3]:
def initCentroids(X, K):

    randidx = np.random.randint(0, X.shape[0], K)
    
    centroids = X[randidx,:]

    return centroids

In [4]:
def assignClusters(X, centroids): 

    K = centroids.shape[0]
    m = X.shape[0]

    C = np.zeros((m,1))
    
    for i in range(m):
        distances = euclidean_distance(X[i,:], centroids)
        C[i] = np.argmin(distances)

    return C

In [5]:
def recenterCentroids(X, C, K):
    
    m,n = X.shape
    
    df = pd.DataFrame(X)
    df.insert(n,"cluster",C)
    
    centroids = df.groupby(by='cluster').mean().values
    
    return centroids

In [6]:
def calculate_cost(X, centroids, cluster):
    # Within-Cluster Sum of Square
    
    C=cluster
    C = C.squeeze()
    cost = 0
    
    K = centroids.shape[0]
    
    for i in range(K):
        cost+= np.sum((X[C==i]-centroids[i])**2)
    
    return cost

In [7]:
def KMeans(X, K):
    
    tic = time.time()
    
    centroids = initCentroids(X, K)
    
    different = True
    i = 0
    centroids_history = []

    while different:
        
        centroids_history.append(centroids)
        
        i+=1

        C = assignClusters(X, centroids)

        centroids = recenterCentroids(X, C, K)
    
        if np.array_equal(centroids_history[-1], centroids) :
            different=False
            
        if i%20 == 0:
            Ct = predict(X, centroids)
            cost = calculate_cost(X, centroids, Ct)
            
            print("Iteration: {} , Cost: {}".format(i, cost))


    Ct = predict(X, centroids)
    cost = calculate_cost(X, centroids, Ct)
            
    print("Iteration: {} , Cost: {}\n".format(i, cost))
      
    toc = time.time()
    print('\nTraining Complete in {} mins after {} iterations'.format(np.round((toc-tic)/60),i))
        
    return centroids

In [8]:
def predict(X_test, centroids):
    
    C = assignClusters(X_test, centroids)
    
    return C

### Select 16 Colors

In [9]:
input_folder = "input_image/"
input_filename = input_folder + os.listdir(input_folder)[0]

In [10]:
img = cv2.imread(input_filename, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [11]:
img = img/255

In [12]:
print("Shape of input image: ", img.shape)
orig_shape = img.shape

Shape of input image:  (1280, 1920, 3)


In [13]:
# Image is reshaped into an m x 3 matrix, where m is the total pixels in the image
# Each row represents 1 pixel

X = np.reshape(img, (img.shape[0] * img.shape[1], 3))

In [14]:
X.shape

(2457600, 3)

In [ ]:
K = 16

centroids = KMeans(X, K)

Iteration: 20 , Cost: 3590.0588507484836
Iteration: 40 , Cost: 3342.4116097798287
Iteration: 60 , Cost: 3215.4512286993704
Iteration: 80 , Cost: 3121.993475359793
Iteration: 100 , Cost: 3069.219164973546
Iteration: 120 , Cost: 3039.8624041460093
Iteration: 140 , Cost: 3016.22677479921


### Compress Image:

In [ ]:
output_folder = "compressed_image/"

In [ ]:
C = predict(X, centroids)

In [ ]:
C = C.astype(np.int8).squeeze()

In [ ]:
def merge_to_byte(C):
    """as the smallest data type in python is of size 1 byte
    The merge_to_byte function takes two 4-bit numbers (actually 1 bytes) as input 
    and combines them into a single byte"""
    
    packed_bytes = []
    
    for i in range(0, C.shape[0], 2):

        n2 = C[i+1] << 4
        packed_byte = n2 | C[i]
        
        packed_bytes.append(packed_byte)
        
    return np.array(packed_bytes, dtype=np.uint8)

In [ ]:
def save_as_RGB(packed_bytes):
    
    m = packed_bytes.shape[0]
    
    dim = np.ceil(np.sqrt(m/3))
    dim = int(dim)
    tot = dim * dim * 3
    pad = int(tot - m)
    
    padding = np.array([1] * pad, dtype=np.uint8)

    padded_packed_bytes = np.append(packed_bytes, padding)
    
    tmp = np.reshape(padded_packed_bytes, (dim, dim, 3))
    
    imageio.imwrite(output_folder+"color_mapping_image_RGB.png", tmp)
    
    return pad

In [ ]:
def save_as_grey(packed_bytes, orig_shape):
    
    l, b, _ = orig_shape
    
    if l > b:
        l = int(l/2)
    else:
        b = int(b/2)
    
    tmp = np.reshape(packed_bytes, (l,b))
    
    imageio.imwrite(output_folder+"color_mapping_image_grey.png", tmp)

In [ ]:
packed_bytes = merge_to_byte(C)

In [ ]:
save_as_grey(packed_bytes, orig_shape)

In [ ]:
np.save(output_folder+"16_colors", centroids)

### Compression Evaluation:

In [ ]:
def get_foldercontent_size(folder_path):
    
    tsize = 0
    
    for file in os.listdir(folder_path):
        tsize += os.path.getsize(folder_path + file)

    return np.round(tsize/1024, 2)

In [ ]:
input_size = get_foldercontent_size(input_folder)

In [ ]:
output_size = get_foldercontent_size(output_folder)

In [ ]:
ratio = ((input_size - output_size) / input_size) * 100
compression_ratio = np.round(ratio, 2)

In [ ]:
cratio = (output_size/input_size)*100
cratio = np.round(cratio, 2)

In [ ]:
result_string = """
Compression Evaluation Results:
--------------------------------

Input Image:
- Filename: {0}
- Size: {1} KB

Output Image (Compressed):
- Filename: {2}
- Size: {3} KB

Compression Details:
- Original Image Size: {1} KB
- Compressed Image Size: {3} KB
- Compression Ratio ≈ {4}%

This represents a compression ratio of approximately {4}%, 
indicating that the compressed image is roughly {5}% of the size of the original image.
""".format(input_filename, input_size, os.listdir(output_folder), output_size, compression_ratio, cratio)

print(result_string)

### Comparsion: Original & Compressed Image

In [ ]:
cm = cv2.imread(output_folder+"color_mapping_image_grey.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
cm = np.reshape(cm, (cm.shape[0] * cm.shape[1], 1)).squeeze()

In [ ]:
def split_byte(cm):
    
    m = len(cm)
    unpacked_bytes = []
    
    for i in range(m):
        
        n2 = cm[i] >> 4
        n1 = cm[i] & 0b00001111
        
        unpacked_bytes.append(n1)
        unpacked_bytes.append(n2)
        
    return unpacked_bytes
    

In [ ]:
unpacked_bytes = split_byte(cm)
C = np.array(unpacked_bytes)

In [ ]:
# Image is recovered from the indices by mapping each pixel to the corresponding cluster centroid

X_recovered = centroids[C.astype(int).tolist(),:]

In [ ]:
# Reshape the recovered image into proper dimensions

# X_recovered = np.reshape(X_recovered, (img.shape[0], img.shape[1], 3))
X_recovered = np.reshape(X_recovered, orig_shape)

In [ ]:
fig, axs = plt.subplots(1,2, squeeze=False, figsize=(15, 15))
axs[0,0].imshow(img)
axs[0,0].set_title("Original Image")
axs[0,1].imshow(X_recovered)
axs[0,1].set_title("Compressed Image")